# setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
from nilearn import image
import nibabel as nib
from importlib import reload

In [ ]:
nb_name = '0_feature_matrices_and_metadata'

curr_dir2 = os.getcwd()
print(curr_dir2)
curr_dir = '/Volumes/ntb/projects/sketchloop02/prototype/link'
os.path.abspath(os.path.join(curr_dir,'..','..'))

## root paths

proj_dir = os.path.abspath(os.path.join(curr_dir,'..','..')) ## use relative paths
feat_dir = os.path.abspath(os.path.join(proj_dir,'data/features')) ## use relative paths 'D:\\data'
data_dir = os.path.abspath(os.path.join(proj_dir,'data'))
reg_file = os.path.abspath(os.path.join(proj_dir,'subjects/{}_neurosketch/regressor/run_{}/{}.txt'))
filt_func = os.path.abspath(os.path.join(proj_dir,'subjects/{}_neurosketch/analysis/firstlevel',
                                         'glm4_recognition_run_{}.feat/filtfuncHIRES.nii.gz'))
#roi_dir = os.path.abspath(os.path.join(proj_dir,'subjects/{}_neurosketch/analysis/firstlevel', 'surfROI'))
roi_dir = '/Volumes/ntb/users/jwammes/sketchloop/subjects/{}_neurosketch/analysis/firstlevel/surfROI'

#metadata_path = '/jukebox/ntb/projects/sketchloop02/data/features/metadata_{}_drawing.csv'

subjects = ['0110171', '0110172', '0111171', '0112171', '0112172', '0112173',
            '0113171', '0115174', '0117171', '0118171', '0118172', '0119171',
            '0119172', '0119173', '0119174', '0120171', '0120172', '0120173',
            '0123171', '0123173', '0124171', '0125171', '0125172', '1121161',
            '1130161', '1202161', '1203161', '1206161', '1206162', '1206163',
            '1207162']
objects = ['bed', 'bench', 'chair', 'table']
copeDict = dict(zip(objects, [1,2,3,4]))

In [ ]:
## add helpers to python path
import sys
if os.path.join(curr_dir2, '..', '..', 'python') not in sys.path:
    sys.path.append(os.path.join(curr_dir2, '..', '..', 'python'))

## module definitions
import object_evidence_analysis_helpers as utils
reload(utils)
utils.data_dir = data_dir
utils.path_to_connect = os.path.join(data_dir, 'features/connect')
#utils.roi_list_connect = roi_list_connect

utils.roi_list_recog = ['V1','V2','LOC','IT','fusiform','parahippo','PRC','ento','hipp']
utils.roi_list_recog_formatted = np.array(['V1', 'V2', 'LOC', 'FUS','PHC','IT','ENT','PRC','HC']) 
roi_list_recog = utils.roi_list_recog
roi_list_masks = ['V1','V2','LOC_FS','IT_FS','fusiform_FS','parahippo_FS','PRC_FS','ento_FS','hipp_FS']

## feature and metadata extraction from recognition runs

In [ ]:
for subject in subjects[:1]:
    for phase in ['12', '34', '56'][:1]:
        # initialize data columns
        subj = [subject] * 160
        label = []
        run_num = [phase[0]]*80 + [phase[1]]*80
        TR_num = []
        
        for rn, run in enumerate(phase):
            # load subject's time series for this run
            #timeseries = image.load_img('{}/{}_neurosketch/analysis/firstlevel/preproc_recognition_run_{}.feat/filtered_func_data.nii.gz'.format(subject, run))
            timeseries = image.load_img(filt_func.format(subject, run))
            # the required reshaping differs depending on which anatomicals are used
            ## Will need to fix up in merged directory
            #timeseries = timeseries.get_data().transpose((3, 0, 1, 2))
            timeseries = timeseries.get_data().transpose((3, 2, 0, 1))
            
            # use information in regressor/run_x folder to make hasImage vector
            # associated TR is just the hasImage index, converted to a float
            Onsets = [0]*240
            for cope in ['bed', 'bench', 'chair', 'table']:
                with open(reg_file.format(subject, r, cope)) as f:
                    times = [line.split(' ')[0] for line in f.read().split('\n')[:-1]]
                    for t in times:
                        TR = int(float(t)/1.5)
                        Onsets[TR] = cope
                        
            # wherever hasImage, we want the features
            features = [timeseries[n+3] for n, onset in enumerate(Onsets) if onset != 0]
            labels = [label for label in Onsets if label != 0]
            FEATURES = np.array(features) if rn == 0 else np.vstack((FEATURES, np.array(features)))
            LABELS = labels if rn == 0 else LABELS + labels
        
        for roi in roi_list_masks:
            mask = nib.load('{}/{}.nii.gz'.format(roi_dir.format(subject), roi))
            maskDat = mask.get_data()
            masked = FEATURES[:, maskDat == 1]
            #np.save('{}/recog/{}_{}_{}_featurematrix.npy'.format(feat_dir, subject, roi, phase), masked)

            ## metadata
            x = pd.DataFrame([subj, LABELS, run_num, TR_num]) # where each of those variables are lists of the same length
            x = x.transpose()
            x.columns = ['subj','label','run_num', 'TR_num']
            #x.to_csv('{}/recog/metadata_{}_{}_{}csv'.format(feat_dir, subject, roi, phase))

## feature and metadata extraction from drawing runs

## creating connectivity features